In [27]:
import numpy as np
import pandas as pd
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [28]:
data = pd.read_csv('daily_minimum_temperature.csv')
data

,Date,Daily minimum temperatures
0,1/1/1981,20.7
1,1/2/1981,17.9
2,1/3/1981,18.8
3,1/4/1981,14.6
4,1/5/1981,15.8
...,...,...
3645,12/27/1990,14
3646,12/28/1990,13.6
3647,12/29/1990,13.5
3648,12/30/1990,15.7


In [29]:
data.isnull().sum()

Date                          0
Daily minimum temperatures    0
dtype: int64

In [30]:
data['Daily minimum temperatures'] = pd.to_numeric(data['Daily minimum temperatures'], errors='coerce')
mean_temp = data['Daily minimum temperatures'].mean()
data['Daily minimum temperatures'].fillna(mean_temp, inplace=True)
threshold_mae = (data['Daily minimum temperatures'].max() - data['Daily minimum temperatures'].min()) * 10/100
print("Threshold MAE: ", threshold_mae)
data.isnull().sum()

Threshold MAE:  2.63


Date                          0
Daily minimum temperatures    0
dtype: int64

In [31]:
train_data, val_data = train_test_split(data, test_size=0.2, shuffle=False)
# Mengambil kolom 'Daily minimum temperatures' untuk training set
train_temp = train_data['Daily minimum temperatures'].values

# Mengambil kolom 'Daily minimum temperatures' untuk validation set
val_temp = val_data['Daily minimum temperatures'].values

In [32]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[-1:]))
    return ds.batch(batch_size).prefetch(1)

In [33]:
train_set = windowed_dataset(train_temp, window_size=60, batch_size=100, shuffle_buffer=1000)
val_set = windowed_dataset(val_temp, window_size=60, batch_size=100, shuffle_buffer=1000)

model = tf.keras.models.Sequential([
  tf.keras.layers.LSTM(60, return_sequences=True, dropout=0.2),
  tf.keras.layers.LSTM(60, dropout=0.2),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
])

In [34]:
class Callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('mae')<threshold_mae and logs.get('val_mae')<threshold_mae):
      print("reached")
      self.model.stop_training = True
callbacks = Callback()

In [35]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1.0000e-04, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])
history = model.fit(train_set, validation_data=val_set, epochs=100, callbacks=[callbacks])

Epoch 1/100
29/29 [==============================] - 10s 167ms/step - loss: 9.9105 - mae: 10.4095 - val_loss: 9.5113 - val_mae: 10.0103
Epoch 2/100
29/29 [==============================] - 4s 137ms/step - loss: 9.1313 - mae: 9.6298 - val_loss: 8.6237 - val_mae: 9.1205
Epoch 3/100
29/29 [==============================] - 5s 175ms/step - loss: 8.2824 - mae: 8.7799 - val_loss: 7.6387 - val_mae: 8.1363
Epoch 4/100
29/29 [==============================] - 4s 138ms/step - loss: 7.2037 - mae: 7.6985 - val_loss: 6.3304 - val_mae: 6.8243
Epoch 5/100
29/29 [==============================] - 4s 137ms/step - loss: 5.7583 - mae: 6.2472 - val_loss: 4.6203 - val_mae: 5.1015
Epoch 6/100
29/29 [==============================] - 5s 172ms/step - loss: 4.1489 - mae: 4.6279 - val_loss: 3.1604 - val_mae: 3.6316
Epoch 7/100
29/29 [==============================] - 4s 137ms/step - loss: 3.2764 - mae: 3.7510 - val_loss: 2.6896 - val_mae: 3.1546
Epoch 8/100
29/29 [==============================] - 4s 129ms/step